In [1]:
# import standard stuff...
import os
import sys
import cellml

First we want to parse the CellML document that we are going to annotate. We parse in non-strict mode to allow any version CellML model to be used.

In [2]:
model_file = 'models/sine_approximations_import.xml'
model = cellml.parse_model(model_file, False)

Now we create an annotator object that we can use to manage the metadata IDs on our model

In [3]:
from libcellml import Annotator
annotator = Annotator()
annotator.setModel(model)

Make sure all entities in the model have an ID and make sure that all IDs in the model are unique.

In [4]:
if annotator.assignAllIds():
    print('Some entities have been assigned an ID, you should save the model!')
else:
    print('Everything already had an ID.')

duplicates = annotator.duplicateIds()
if len(duplicates) > 0:
    print("There are some duplicate IDs, behaviour may be unreliable...")
    print(duplicates)

Some entities have been assigned an ID, you should save the model!


Since this test model has some messed up IDs, lets just blow them away and start again...

In [5]:
annotator.clearAllIds()
annotator.assignAllIds()
model_string = cellml.print_model(model)
print(model_string)

# and save the updated model to a new file - note, we need the model filename for making our annotations later
model_file = 'models/sine_approximations-updated-ids.xml'
with open(model_file, 'w') as f:
    f.write(model_string)

<?xml version="1.0" encoding="UTF-8"?>
<model xmlns="http://www.cellml.org/cellml/2.0#" name="sin_approximations_import" id="b4da7a">
  <import xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="sin.xml" id="b4da7b">
    <component component_ref="sin" name="actual_sin" id="b4da7f"/>
  </import>
  <import xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="deriv_approx_sin.xml" id="b4da7c">
    <component component_ref="sin" name="deriv_approx_sin" id="b4da80"/>
  </import>
  <import xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="parabolic_approx_sin.xml" id="b4da7d">
    <component component_ref="sin" name="parabolic_approx_sin" id="b4da81"/>
  </import>
  <component name="main" id="b4da7e">
    <variable name="x" units="dimensionless" interface="public_and_private" id="b4da82"/>
    <variable name="sin1" units="dimensionless" interface="public_and_private" id="b4da83"/>
    <variable name="sin2" units="dimensionless" interface="public_and_private" id="b4da84"/>
    <var

Now we can do some annotation using pyOmexMeta (see https://github.com/sys-bio/pyomexmeta-binder-notebook) - except it doesn't work for CellML 2.0 models. So for now we work directly with RDF.

In [8]:
# get the ID of the variables we want to annotate
actual_sine_variable = model.component('main').variable('sin1').id()
independent_variable = model.component('main').variable('x').id()

output_variables = [model.component('main').variable('sin1').id(), 
                    model.component('main').variable('sin2').id(), 
                    model.component('main').variable('sin3').id(), 
                    model.component('main').variable('x').id()]

# make an RDF Graph to add annotations to - using rdflib
from rdflib import Graph, Literal, RDF, URIRef
from rdflib.namespace import DCTERMS

g = Graph()

# define some URIs for things we need

# use this URI to identify delayed variables - not the perfect URI, but will do for now
#     This is actually "delayed differential equation model" from the MAMO ontology
#delay_variable_uri = URIRef('http://identifiers.org/mamo/MAMO_0000089')
delay_variable_uri = URIRef('https://github.com/nickerso/libcellml-python-utils/properties.rst#delay-variable')
variable_to_delay_uri = URIRef('https://github.com/nickerso/libcellml-python-utils/properties.rst#variable-to-delay')

# use this for some random thing that I want to define - http://example.com is a good base for things that will never resolve
stuff_uri = URIRef('http://example.com/cool-thing#21')

# a "readout" variable that we maybe want to connect to something external?
timecourse_readout_uri = URIRef('http://identifiers.org/mamo/MAMO_0000031')

# Create an RDF URI node for our variable to use as the subject for multiple triples
# note: we are going to serialise the RDF graph into the same folder, so we need a URI that is relative to the intended file
variable_uri = URIRef('sine_approximations-updated-ids.xml' + '#' + actual_sine_variable)
independent_variable_uri = URIRef('sine_approximations-updated-ids.xml' + '#' + independent_variable)

# Add triples using store's add() method.
# We're using the Dublin Core term "type" to associate the variable with the delay...
g.add((variable_uri, DCTERMS.type, variable_to_delay_uri))
g.add((variable_uri, delay_variable_uri, independent_variable_uri))
# Set timecourse readout variables
for ov in output_variables:
    output_uri = URIRef('sine_approximations-updated-ids.xml' + '#' + ov)
    g.add((output_uri, DCTERMS.type, timecourse_readout_uri))

# print all the data in the turtle format
print(g.serialize(format='ttl'))

# and save to a file
with open('models/sine_approximations-updated-ids--annotations.ttl', 'w') as f:
    f.write(g.serialize(format='ttl'))

@prefix ns1: <http://purl.org/dc/terms/> .
@prefix ns2: <https://github.com/nickerso/libcellml-python-utils/properties.rst#> .

<sine_approximations-updated-ids.xml#b4da83> ns1:type <http://identifiers.org/mamo/MAMO_0000031>,
        ns2:variable-to-delay ;
    ns2:delay-variable <sine_approximations-updated-ids.xml#b4da82> .

<sine_approximations-updated-ids.xml#b4da84> ns1:type <http://identifiers.org/mamo/MAMO_0000031> .

<sine_approximations-updated-ids.xml#b4da86> ns1:type <http://identifiers.org/mamo/MAMO_0000031> .

<sine_approximations-updated-ids.xml#b4da82> ns1:type <http://identifiers.org/mamo/MAMO_0000031> .


